# 🧠 Local RAG QA Bot (Ollama + LangChain + ChromaDB)
This notebook runs a local Retrieval-Augmented Generation (RAG) chatbot using open-source components — no API keys, no internet required.

**Components used:**
- LangChain
- ChromaDB
- Ollama (local LLM runner)
- sentence-transformers for embeddings

**Tested on macOS (M-series & Intel) with Python 3.10+.**

In [ ]:
!pip install langchain chromadb pypdf sentence-transformers ollama --quiet

In [ ]:
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import Ollama
import os
print('✅ Libraries loaded successfully')

## Step 1 — Load your document(s)
You can provide a path to any PDF or `.txt` file(s).

In [ ]:
file_path = input('Enter path to your document (PDF or TXT): ').strip()

if file_path.endswith('.pdf'):
    loader = PyPDFLoader(file_path)
elif file_path.endswith('.txt'):
    loader = TextLoader(file_path)
else:
    raise ValueError('Please provide a .pdf or .txt file.')

documents = loader.load()
print(f'✅ Loaded {len(documents)} pages from {file_path}')

## Step 2 — Split into chunks & create embeddings

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = splitter.split_documents(documents)
print(f'✅ Split into {len(texts)} chunks')

embedding_model = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
vectordb = Chroma.from_documents(texts, embedding=embedding_model, persist_directory='db')
retriever = vectordb.as_retriever(search_kwargs={'k': 3})
print('✅ Vector database built and ready!')

## Step 3 — Initialize Local Ollama Model

In [ ]:
model_name = input('Enter Ollama model name (default: llama3): ').strip() or 'llama3'
llm = Ollama(model=model_name)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
print(f'✅ Using local Ollama model: {model_name}')

## Step 4 — Ask Questions

In [ ]:
print('💬 Ask questions about your document. Type exit() to quit.')
while True:
    query = input('Question: ')
    if query.lower() in ['exit', 'quit', 'q']:
        print('👋 Goodbye!')
        break
    answer = qa_chain.run(query)
    print('Answer:', answer)